# 1. Разметка

In [6]:
from nltk.tokenize import NLTKWordTokenizer
import json

In [7]:
tokenizer = NLTKWordTokenizer()

In [8]:
text_1 = '''Не прилично ли будет нам, братия,
Начать древним складом
Печальную повесть о битвах Игоря,
Игоря Святославича!
Начаться же сей песни
По былинам сего времени,
А не по вымыслам Бояновым.
Вещий Боян,
Если песнь кому сотворить хотел,
Растекался мыслию по древу,
Серым волком по земли,
Сизым орлом под облаками.'''

text_2 = '''Вашу мысль,
мечтающую на размягченном мозгу,
как выжиревший лакей на засаленной кушетке,
буду дразнить об окровавленный сердца лоскут:
досыта изъиздеваюсь, нахальный и едкий.
У меня в душе ни одного седого волоса,
и старческой нежности нет в ней!
Мир огромив мощью голоса,
иду — красивый,
двадцатидвухлетний.
Нежные!
Вы любовь на скрипки ложите.
Любовь на литавры ложит грубый.
А себя, как я, вывернуть не можете,
чтобы были одни сплошные губы!
Приходите учиться —
из гостиной батистовая,
чинная чиновница ангельской лиги.'''

text_3 = '''Низкий дом без меня ссутулится,
Старый пёс мой давно издох.
На московских изогнутых улицах
Умереть, знать, сулил мне Бог.
Я люблю этот город вязевый,
Пусть обрюзг он и пусть одрях.
Золотая дремотная Азия
Опочила на куполах.
А когда ночью светит месяц,
Когда светит… чёрт знает как!
Я иду, головою свесясь,
Переулком в знакомый кабак.
Шум и гам в этом логове жутком,
Но всю ночь напролёт, до зари,
Я читаю стихи проституткам
И с бандитами жарю спирт.'''

text_4 = '''Седое в джинсах от кутюр
Абсурд, карикатура
Смешно... в стране мультикультур
Закончилась культура
И свет погас, и гроб готов
И мне клауд-грустно
В стране есенинских стихов
Закончилось искусство
Рояльчик в парке городском
Расстроен очень-очень
И кто-то страшный жмёт на нём
Бемоль чернее ночи
И в чёрном худи бродит там
Провинциальный Бэнкси
Баллоном строчки по углам
Слова из этой песни'''
corpus = [text_1, text_2, text_3, text_4]

Я выбрал 4 очень разных текста с разными особенностями:<br>
* В первом тексте сложность представляют нерегулярные формы (братия, песнь, мыслию, [по] земли)
* Во втором тексте много редких форм (выжиревший, изъиздеваюсь) и авторчких слов (огромив), а также нелитературные формы (ложите)
* В третьем тексте встречаются редкие глагольные лексемы (обрюзг, опочила)
* В четвёртом тексте присутствуют авторские слова (клауд-грустно), редкие лексемы (мультикультур, есенинских), а также нетривиальные синтаксические структуры (Баллоном строчки по углам / Слова из этой песни)

Далее я вручную производил разметку в формате словаря: 
```python
{номер токена: 
                 {слово: 'слово', возможные_тэги: 
                                                  (тэг_1, тэг_2,...)
                  }
}
```

<b>Это был код разметки</b><br>
tagged = {} <br>
i = 0<br>
for text in corpus:<br>
    for token in tokenizer.tokenize(text.lower()):<br>
        if token.strip('.,!?:…—') != '':<br>
            word = token.strip(".,!?:…—")<br>
            tags = tuple(input(f'{i}. POS tag of "{word}": ').split(' '))<br>
            tagged[i] = {'word': word, 'tags': tags}<br>
            i += 1

При разметке я пользовался указаниями из домашней статьи, так как их целью было избежать неоднозначных моментов, а кроме того, к ней легко сводимы другие типы разметки. Разметка выглядит вот так [Ляшевская и др. 2010]:<br>
* существительные (S)<br>
* прилагательные (A)<br>
* глаголы, в том числе причастия и деепричастия (V)<br>
* предлоги (PR)<br>
* союзы (CONJ)<br>
* сборная категория, включающая прочие несклоняемые слова: наречия, вводные слова, частицы, междометия (ADV)<br>
<br>
Не участвовали в оценке и могли быть размечены любым образом местоимения (включая наречные и предикативные), числительные, а также составные предлоги и союзы (ср. потому что, в течение).

<b>На этом этапе я записал свою разметку в json файл</b><br>
with open('tagged.json', 'w', encoding='utf-8') as f: <br>
    json.dump(tagged, f, ensure_ascii=False, indent='\t')

In [73]:
with open('tagged.json', 'r', encoding='utf-8') as f: # достаю свою разметку из файла
    tagged = json.load(f)

In [10]:
for i in range(259): # перевожу индексы токенов из строк в числа
    tagged[i] = tagged[str(i)]
    tagged.pop(str(i))

In [11]:
len(tagged)

259

# 2. Тэггеры

## 2.1. pymorphy

In [12]:
from pymorphy2 import MorphAnalyzer

Создаю словарь-конвертер

In [13]:
pymorphy_conv = {
    'NOUN': 'S',
    'ADJF': 'A',
    'ADJS': 'A',
    'COMP': 'A',
    'VERB': 'V',
    'INFN': 'V',
    'PRTF': 'V',
    'PRTS': 'V',
    'GRND': 'V',
    'NUMR': '-',
    'ADVB': 'ADV',
    'NPRO': '-',
    'PRED': 'ADV',
    'PREP': 'PR',
    'CONJ': 'CONJ',
    'PRCL': 'ADV',
    'INTJ': 'ADV'
}

In [14]:
m = MorphAnalyzer()

Токенизирую и определяю части речи

In [15]:
pymorphy_tagged = {}
i = 0
for text in corpus:
    for token in tokenizer.tokenize(text.lower()):
        if token.strip('.,!?:…—') != '':
            pos = m.parse(token.strip('.,!?:…'))[0].tag.POS
            if pos != None:
                pymorphy_tagged[i] = pymorphy_conv[pos]
            else:
                pymorphy_tagged[i] = '-'
            i += 1

In [16]:
pymorphy_tagged

{0: 'ADV',
 1: 'ADV',
 2: 'CONJ',
 3: 'V',
 4: '-',
 5: 'S',
 6: 'V',
 7: 'A',
 8: 'S',
 9: 'A',
 10: 'S',
 11: 'PR',
 12: 'S',
 13: 'S',
 14: 'S',
 15: 'S',
 16: 'V',
 17: 'ADV',
 18: 'A',
 19: 'S',
 20: 'PR',
 21: 'S',
 22: '-',
 23: 'S',
 24: 'CONJ',
 25: 'ADV',
 26: 'PR',
 27: 'S',
 28: 'S',
 29: 'A',
 30: 'S',
 31: 'CONJ',
 32: 'S',
 33: '-',
 34: 'V',
 35: 'V',
 36: 'V',
 37: 'S',
 38: 'PR',
 39: 'S',
 40: 'A',
 41: 'S',
 42: 'PR',
 43: 'S',
 44: 'A',
 45: 'S',
 46: 'PR',
 47: 'S',
 48: 'A',
 49: 'S',
 50: 'V',
 51: 'PR',
 52: 'V',
 53: 'S',
 54: 'CONJ',
 55: 'V',
 56: 'S',
 57: 'PR',
 58: 'A',
 59: 'S',
 60: 'V',
 61: 'V',
 62: 'PR',
 63: 'A',
 64: 'S',
 65: 'S',
 66: 'ADV',
 67: 'V',
 68: 'A',
 69: 'CONJ',
 70: 'A',
 71: 'PR',
 72: '-',
 73: 'PR',
 74: 'S',
 75: 'ADV',
 76: 'A',
 77: 'A',
 78: 'S',
 79: 'CONJ',
 80: 'A',
 81: 'S',
 82: 'ADV',
 83: 'PR',
 84: '-',
 85: 'S',
 86: 'V',
 87: 'S',
 88: 'S',
 89: 'V',
 90: 'A',
 91: 'A',
 92: 'A',
 93: '-',
 94: 'S',
 95: 'PR',
 96: 

Длина совпадает с длиной размеченных токенов

In [17]:
len(pymorphy_tagged)

259

## 2.2. mystem

In [18]:
from pymystem3 import Mystem

Создаю словарь-конвертер

In [19]:
mystem_conv = {
    'A': 'A',
    'ADV': 'ADV',
    'ADVPRO': 'ADV',
    'ANUM': '-',
    'APRO': '-',
    'COM': '-',
    'CONJ': 'CONJ',
    'INTJ': 'ADV',
    'NUM': '-',
    'PART': 'ADV',
    'PR': 'PR',
    'S': 'S',
    'SPRO': '-',
    'V': 'V'
}

In [20]:
s = Mystem()

In [21]:
mystem_tagged = {}
i = 0
for text in corpus:
    for word in s.analyze(text):
        if word['text'].isalpha():    
            pos = word['analysis'][0]['gr'].split(',')[0].split('=')[0]
            mystem_tagged[i] = {'tag': mystem_conv[pos], 'word': word['text']}
            i += 1

In [22]:
mystem_tagged

{0: {'tag': 'ADV', 'word': 'Не'},
 1: {'tag': 'ADV', 'word': 'прилично'},
 2: {'tag': 'ADV', 'word': 'ли'},
 3: {'tag': 'V', 'word': 'будет'},
 4: {'tag': '-', 'word': 'нам'},
 5: {'tag': 'S', 'word': 'братия'},
 6: {'tag': 'V', 'word': 'Начать'},
 7: {'tag': 'A', 'word': 'древним'},
 8: {'tag': 'S', 'word': 'складом'},
 9: {'tag': 'A', 'word': 'Печальную'},
 10: {'tag': 'S', 'word': 'повесть'},
 11: {'tag': 'PR', 'word': 'о'},
 12: {'tag': 'S', 'word': 'битвах'},
 13: {'tag': 'S', 'word': 'Игоря'},
 14: {'tag': 'S', 'word': 'Игоря'},
 15: {'tag': 'S', 'word': 'Святославича'},
 16: {'tag': 'V', 'word': 'Начаться'},
 17: {'tag': 'ADV', 'word': 'же'},
 18: {'tag': '-', 'word': 'сей'},
 19: {'tag': 'S', 'word': 'песни'},
 20: {'tag': 'PR', 'word': 'По'},
 21: {'tag': 'S', 'word': 'былинам'},
 22: {'tag': '-', 'word': 'сего'},
 23: {'tag': 'S', 'word': 'времени'},
 24: {'tag': 'CONJ', 'word': 'А'},
 25: {'tag': 'ADV', 'word': 'не'},
 26: {'tag': 'PR', 'word': 'по'},
 27: {'tag': 'S', 'word

Длина полученного словаря получилась больше, чем размеченного вручную, нужно убрать один токен

In [23]:
len(mystem_tagged)

260

Оеазывается, он разбил "клауд-грустно" на два токена. Я решил выкинуть часть "клауд"

In [24]:
mystem_tagged.pop(221)

{'tag': 'S', 'word': 'клауд'}

Изменяю индексы токенов, чтобы они сошлись

In [25]:
for i in range(221, 259):
    mystem_tagged[i] = mystem_tagged[i+1]

Последний токен теперь дублируется, выкидываю его

In [26]:
mystem_tagged.pop(259)

{'tag': 'S', 'word': 'песни'}

Теперь размеры совпадают

In [27]:
mystem_tagged

{0: {'tag': 'ADV', 'word': 'Не'},
 1: {'tag': 'ADV', 'word': 'прилично'},
 2: {'tag': 'ADV', 'word': 'ли'},
 3: {'tag': 'V', 'word': 'будет'},
 4: {'tag': '-', 'word': 'нам'},
 5: {'tag': 'S', 'word': 'братия'},
 6: {'tag': 'V', 'word': 'Начать'},
 7: {'tag': 'A', 'word': 'древним'},
 8: {'tag': 'S', 'word': 'складом'},
 9: {'tag': 'A', 'word': 'Печальную'},
 10: {'tag': 'S', 'word': 'повесть'},
 11: {'tag': 'PR', 'word': 'о'},
 12: {'tag': 'S', 'word': 'битвах'},
 13: {'tag': 'S', 'word': 'Игоря'},
 14: {'tag': 'S', 'word': 'Игоря'},
 15: {'tag': 'S', 'word': 'Святославича'},
 16: {'tag': 'V', 'word': 'Начаться'},
 17: {'tag': 'ADV', 'word': 'же'},
 18: {'tag': '-', 'word': 'сей'},
 19: {'tag': 'S', 'word': 'песни'},
 20: {'tag': 'PR', 'word': 'По'},
 21: {'tag': 'S', 'word': 'былинам'},
 22: {'tag': '-', 'word': 'сего'},
 23: {'tag': 'S', 'word': 'времени'},
 24: {'tag': 'CONJ', 'word': 'А'},
 25: {'tag': 'ADV', 'word': 'не'},
 26: {'tag': 'PR', 'word': 'по'},
 27: {'tag': 'S', 'word

# 2.3. spacy

In [28]:
import spacy
nlp = spacy.load("ru_core_news_lg")

Создаю словарь-конвертер

In [29]:
spacy_conv = {
    'ADJ': 'A',
    'ADP': 'PR',
    'ADV': 'ADV',
    'AUX': 'V',
    'CCONJ': 'CONJ',
    'DET': '-',
    'INTJ': 'ADV',
    'NOUN': 'S',
    'NUM': '-',
    'PART': 'ADV',
    'PRON': '-',
    'PROPN': 'S',
    'PUNCT': '-',
    'SCONJ': 'CONJ',
    'SYM': '-',
    'VERB': 'V',
    'X': '-'
}

Здесь также не провожу токенизации из-за наличия встроенного токенизатора

In [30]:
spacy_tagged = {}
i = 0
for text in corpus:
    for word in nlp(text):
        if word.text.isalpha(): 
            spacy_tagged[i] = {'tag': spacy_conv[word.pos_], 'word': word.text}
            i += 1

In [31]:
spacy_tagged

{0: {'tag': 'ADV', 'word': 'Не'},
 1: {'tag': 'A', 'word': 'прилично'},
 2: {'tag': 'ADV', 'word': 'ли'},
 3: {'tag': 'V', 'word': 'будет'},
 4: {'tag': '-', 'word': 'нам'},
 5: {'tag': 'S', 'word': 'братия'},
 6: {'tag': 'V', 'word': 'Начать'},
 7: {'tag': 'A', 'word': 'древним'},
 8: {'tag': 'S', 'word': 'складом'},
 9: {'tag': 'A', 'word': 'Печальную'},
 10: {'tag': 'S', 'word': 'повесть'},
 11: {'tag': 'PR', 'word': 'о'},
 12: {'tag': 'S', 'word': 'битвах'},
 13: {'tag': 'S', 'word': 'Игоря'},
 14: {'tag': 'S', 'word': 'Игоря'},
 15: {'tag': 'S', 'word': 'Святославича'},
 16: {'tag': 'V', 'word': 'Начаться'},
 17: {'tag': 'ADV', 'word': 'же'},
 18: {'tag': '-', 'word': 'сей'},
 19: {'tag': 'S', 'word': 'песни'},
 20: {'tag': 'PR', 'word': 'По'},
 21: {'tag': 'S', 'word': 'былинам'},
 22: {'tag': '-', 'word': 'сего'},
 23: {'tag': 'S', 'word': 'времени'},
 24: {'tag': 'CONJ', 'word': 'А'},
 25: {'tag': 'ADV', 'word': 'не'},
 26: {'tag': 'PR', 'word': 'по'},
 27: {'tag': 'S', 'word':

Получилось на 3 токена больше, чем в ручной разметке

In [32]:
len(spacy_tagged)

262

Здесь токенизатор также разделил "клауд-грустно", а ещё "очень-очень" и "кто-то". Избавляюсь от "клауд", "очень" и "то" (в последнем случае это не повлияет на результат, так как местоимения и так не входят в оценку)

In [33]:
spacy_tagged.pop(221)
spacy_tagged.pop(235)
spacy_tagged.pop(238)

{'tag': '-', 'word': 'то'}

Перераспределяю индексы

In [34]:
for i in range(221,234):
    spacy_tagged[i] = spacy_tagged[i+1]
for i in range(234, 236):
    spacy_tagged[i] = spacy_tagged[i+2]
for i in range(236, 259):
    spacy_tagged[i] = spacy_tagged[i+3]

Выкидываю продублировавшийся токен

In [35]:
spacy_tagged.pop(259)
spacy_tagged.pop(260)
spacy_tagged.pop(261)

{'tag': 'S', 'word': 'песни'}

In [36]:
spacy_tagged

{0: {'tag': 'ADV', 'word': 'Не'},
 1: {'tag': 'A', 'word': 'прилично'},
 2: {'tag': 'ADV', 'word': 'ли'},
 3: {'tag': 'V', 'word': 'будет'},
 4: {'tag': '-', 'word': 'нам'},
 5: {'tag': 'S', 'word': 'братия'},
 6: {'tag': 'V', 'word': 'Начать'},
 7: {'tag': 'A', 'word': 'древним'},
 8: {'tag': 'S', 'word': 'складом'},
 9: {'tag': 'A', 'word': 'Печальную'},
 10: {'tag': 'S', 'word': 'повесть'},
 11: {'tag': 'PR', 'word': 'о'},
 12: {'tag': 'S', 'word': 'битвах'},
 13: {'tag': 'S', 'word': 'Игоря'},
 14: {'tag': 'S', 'word': 'Игоря'},
 15: {'tag': 'S', 'word': 'Святославича'},
 16: {'tag': 'V', 'word': 'Начаться'},
 17: {'tag': 'ADV', 'word': 'же'},
 18: {'tag': '-', 'word': 'сей'},
 19: {'tag': 'S', 'word': 'песни'},
 20: {'tag': 'PR', 'word': 'По'},
 21: {'tag': 'S', 'word': 'былинам'},
 22: {'tag': '-', 'word': 'сего'},
 23: {'tag': 'S', 'word': 'времени'},
 24: {'tag': 'CONJ', 'word': 'А'},
 25: {'tag': 'ADV', 'word': 'не'},
 26: {'tag': 'PR', 'word': 'по'},
 27: {'tag': 'S', 'word':

Теперь длина сходится

In [37]:
len(spacy_tagged)

259

# 3. Измерение качества

## 3.1. pymorphy

In [38]:
right = 0
wrong = 0
for num, val in tagged.items():
    if val['tags'][0] != '-': # В оценке не участвуют токены, отмеченные как "-"
        if pymorphy_tagged[num] in val['tags']: 
            # Если тэг автоматической разметки есть среди возможных тэгов, то результат засчитывается как верный, 
            # иначе - неверный
            right += 1
        else:
            wrong += 1

In [39]:
accuracy = right / (right+wrong)
accuracy

0.9828326180257511

## 3.2. mystem

In [40]:
right = 0
wrong = 0
for num, val in tagged.items():
    if val['tags'][0] != '-':
        if mystem_tagged[num]['tag'] in val['tags']:
            right += 1
        else:
            wrong += 1

In [41]:
accuracy = right / (right+wrong)
accuracy

0.9785407725321889

## 3.3. spacy

In [42]:
right = 0
wrong = 0
for num, val in tagged.items():
    if val['tags'][0] != '-':
        if spacy_tagged[num]['tag'] in val['tags']:
            right += 1
        else:
            wrong += 1

In [43]:
accuracy = right / (right+wrong)
accuracy

0.9399141630901288

Итого, лучше всего справился pymorphy, с ним и будем работать дальше

# 4. Чанкер

## 4.1. Шаблоны

Шаблон 1: плохо + V, как маркер отрицательного отзыва, потому что "плохо работает", "плохо справился [с задачей]" итд.<br>
Шаблон 2: осторожно + S, как маркер отрицательного отзыва, так как обычно "осторожно мошенники".<br>
Шаблон 3: достаточно + A, как маркер положительного отзыва, потому что "достаточно хороший", "достаточно быстрый" (причём именно в положительном ключе) итд.

## 4.2. Создание чанкера

In [53]:
def chunker(text):
    positive = 0
    negative = 0
    tokens = tokenizer.tokenize(text.lower()) # токенизирую
    for i in range(len(tokens)):
        word = tokens[i].strip('.,!?:…—') # чищу токены
        # 1. ищу первый шаблоон, за каждый найденный шаблон, добавляю балл к негативной оценке
        if word == 'плохо' and i < len(tokens) - 1:
            next_word = tokens[i+1].strip('.,!?:…—')
            if m.parse(next_word)[0].tag.POS != None: 
                # проверяю, не является ли токен, идущий сразу за первым пунктуацией или другим мусором,
                # если нет, то рассматриваю его, если является, то смотрю на следующий токен
                if pymorphy_conv[m.parse(next_word)[0].tag.POS] == 'V':
                    negative += 1
            elif i < len(tokens) - 2:
                second_next = tokens[i+2].strip('.,!?:…—')
                if m.parse(second_next)[0].tag.POS != None and \
                                                        pymorphy_conv[m.parse(second_next)[0].tag.POS] == 'V':
                    negative += 1
        # 2. ищу второй шаблоон, за каждый найденный шаблон, добавляю балл к негативной оценке (аналогично)
        if word == 'осторожно' and i < len(tokens) - 1:
            next_word = tokens[i+1].strip('.,!?:…—')
            if m.parse(next_word)[0].tag.POS != None:
                if pymorphy_conv[m.parse(next_word)[0].tag.POS] == 'S':
                    negative += 1
            elif i < len(tokens) - 2:
                second_next = tokens[i+2].strip('.,!?:…—')
                if m.parse(second_next)[0].tag.POS != None and \
                                                    pymorphy_conv[m.parse(second_next)[0].tag.POS] == 'S':
                    negative += 1
        # 3. ищу третий шаблоон, за каждый найденный шаблон, добавляю балл к положительной оценке (аналогично)
        if word == 'довольно' and i < len(tokens) - 1:
            next_word = tokens[i+1].strip('.,!?:…—')
            if m.parse(next_word)[0].tag.POS != None:
                if pymorphy_conv[m.parse(next_word)[0].tag.POS] == 'A':
                    positive += 1
            elif i < len(tokens) - 2:
                second_next = tokens[i+2].strip('.,!?:…—')
                if m.parse(second_next)[0].tag.POS != None and \
                                                    pymorphy_conv[m.parse(second_next)[0].tag.POS] == 'A':
                    positive += 1
    # возвращаю разницу между позитивной оценкой и негативной
    return positive-negative

## 4.3. Встраивание чанкера

In [71]:
# в функцию из предыдущего домашнего задания добавляю в подсчёте оценки оценку чанкера, если activate_chunker=True
def tone_pred(texts, pos_set, neg_set, activate_chunker=False):
    y_pred = []
    for text in texts:
        val = 0
        for token in casual_tokenize(text, preserve_case=False): #токенизирую отзыв, тональность которого угадывается
            features = m.parse(token)[0] 
            if not(features.tag.POS): #не учитываю пунктуацию и мусор
                continue
            lemma = features.normal_form #лемматиизирую
            if lemma in pos_set: #если слово находится в положительном списке, то к оценке тональности прибавляю 1
                val += 1
            if lemma in neg_set: #если слово находится в отрицательном списке, то от оценки тональности отнимаю 1
                val -= 1
        if activate_chunker:
            val += chunker(text)
        if val > 0: #смотрю на знак оценки тональности и по нему делаю предсказание
            y_pred.append(1)
        if val < 0:
            y_pred.append(-1)
        if val == 0:
            y_pred.append(0)
    return(y_pred)

## 4.4. Проверка качества

Здесь я прогоняю код подготовки словарей и тестовой выборки из предыдущего домашнего задания. Только здесь я увеличил размер тестовой выборки до 40 отзывов

In [68]:
import numpy as np
import random

with open('reviews.json', 'r', encoding='utf-8') as f:
    reviews = json.load(f)

random.seed(1)
len_neg = np.array([len(i) for i in reviews['negative']])
reviews['negative'] = random.choices(np.array(reviews['negative'])[len_neg > 535], k=100)
test = reviews['negative'][-20:]
reviews['negative'] = reviews['negative'][:-20]
test.extend(reviews['positive'][-20:])
reviews['positive'] = reviews['positive'][:-20]
y = [-1 for i in range(20)]
y.extend([1 for i in range(20)])

from nltk.tokenize import casual_tokenize
neg_prep = []
pos_prep = []
for review in reviews['negative']:
    neg_prep.append(casual_tokenize(review, preserve_case=False))
for review in reviews['positive']:
    pos_prep.append(casual_tokenize(review, preserve_case=False))
    
neg_lemma = []
pos_lemma = []
for review in neg_prep:
    rev_lemma = []
    for word in review:
        features = m.parse(word)[0]
        if not(features.tag.POS): #на этом моменте я избавляюсь от пунктуации и мусора
            continue
        rev_lemma.append(features.normal_form)
    neg_lemma.append(rev_lemma)

for review in pos_prep:
    rev_lemma = []
    for word in review:
        features = m.parse(word)[0]
        if not(features.tag.POS):
            continue
        rev_lemma.append(features.normal_form)
    pos_lemma.append(rev_lemma)
    
from collections import Counter
neg_count = Counter([word for review in neg_lemma for word in review])
pos_count = Counter([word for review in pos_lemma for word in review])
neg_words = np.array([i for i in neg_count.keys()])
neg_vals = np.array([i for i in neg_count.values()])
pos_words = np.array([i for i in pos_count.keys()])
pos_vals = np.array([i for i in pos_count.values()])
neg_set = set(neg_words[neg_vals > 4])
pos_set = set(pos_words[pos_vals > 4])
unique_neg = neg_set.difference(pos_set)
unique_pos = pos_set.difference(neg_set)

Измеряю качество без чанкера

In [69]:
from sklearn.metrics import accuracy_score
accuracy_score(y, tone_pred(test, unique_pos, unique_neg))

0.55

И с чанкером

In [72]:
accuracy_score(y, tone_pred(test, unique_pos, unique_neg, activate_chunker=True))

0.575

С чанкером получилось немного лучше. Дело в том, что такие шаблоны могут встречаться довольно нечасто, и чтобы увидеть значительное улучшение, нужно либо увеличить их количество, либо увеличить размер тестовой выборки